<a href="https://colab.research.google.com/github/pedrolustosab/medlineplus/blob/main/medlineplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Importando Bibliotecas
import requests
import re
from bs4 import BeautifulSoup

In [8]:
# URL
infectiousdiseases_url = "https://medlineplus.gov/infections.html"

In [14]:
# Requisição
try:
    response = requests.get(infectiousdiseases_url)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    print('Erro ao fazer a requisição:', e)
    exit()

infectiousdiseases_html = BeautifulSoup(response.content, 'html.parser')

In [16]:
# Encontrar todos os elementos com a classe "item"
items = infectiousdiseases_html.find_all(class_='item')

# Inicializar uma lista para armazenar as doenças
diseases = []

# Exibir os textos dos elementos <a> dentro dos elementos com a classe "item"
for item in items:
    # Encontrar todos os elementos <a> dentro do elemento com a classe "item"
    a_tags = item.find_all('a')
    for a_tag in a_tags:
        disease = a_tag.text.strip()
        if disease != 'Animal Diseases and Your Health':
            diseases.append(disease)

In [28]:
# Dicionário para armazenar as doenças e sintomas
disease_symptoms = {}

# Itera sobre cada doença na lista de doenças
for disease in diseases:
    # Limpa o nome da doença para criar a URL
    disease_clear = re.sub("[^\w]", "", disease.lower())
    disease_html = f'https://medlineplus.gov/{disease_clear}.html'

    try:
        # Tenta fazer uma requisição HTTP para obter o conteúdo da página da doença
        response = requests.get(disease_html)
        response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
    except requests.exceptions.RequestException as e:
        # Em caso de erro na requisição, exibe uma mensagem de erro e encerra o programa
        print('Erro ao fazer a requisição:', e)
        exit()

    # Analisa o conteúdo HTML da página da doença
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontra o primeiro elemento <h3> que contém "What are the symptoms of"
    symptoms_header = soup.find('h3', string=re.compile(r'What are the symptoms of'))

    if not symptoms_header:
        # Se não houver um cabeçalho de sintomas, adiciona uma lista vazia de sintomas
        symptoms = []
    else:
        # Adiciona uma lista vazia de sintomas
        symptoms = []

        # Encontrar o próximo elemento <h3>
        last_element = symptoms_header.find_next_sibling('h3')

        # Enquanto não chegar ao próximo elemento <h3>
        while symptoms_header != last_element:
            # Avança para o próximo elemento irmão
            symptoms_header = symptoms_header.find_next_sibling()
            #Verifica se existe <li>
            elements = symptoms_header.find_all('li')
            if elements:
                # Adiciona cada item da lista de sintomas ao resultado final
                for element in elements:
                    symptoms.extend([item.strip() for item in element.text.splitlines() if item.strip()])

    # Adiciona a doença e seus sintomas ao dicionário disease_symptoms
    disease_symptoms[disease] = symptoms


In [33]:
# Cria uma lista com as chaves do dicionário disease_symptoms cujos valores são listas vazias
empty_symptoms_keys = [key for key, value in disease_symptoms.items() if not value]

# Imprime o número de chaves com valores vazios
print(len(empty_symptoms_keys))


90


In [32]:
empty_symptoms_keys

['Aspergillosis',
 'Abscess',
 'Acute Bronchitis',
 'Viral Infections',
 'Vaccines',
 'HIV and Infections',
 'Anthrax',
 'Antibiotic Resistance',
 'Antibiotics',
 'Meningitis',
 "Athlete's Foot",
 'Tick Bites',
 'Bacterial Infections',
 'Urinary Tract Infections',
 'Infection Control',
 'Body Lice',
 'Germs and Hygiene',
 'Bone Infections',
 'Botulism',
 'Plague',
 'C. diff Infections',
 'Campylobacter Infections',
 'Yeast Infections',
 'Cat Scratch Disease',
 'Cellulitis',
 'Chickenpox',
 'Chikungunya',
 'Childhood Vaccines',
 'Cholera',
 'Gonorrhea',
 'Cleaning, Disinfecting, and Sanitizing',
 'Cytomegalovirus Infections',
 'Valley Fever',
 'Cold Sores',
 'Infectious Diseases',
 'Genital Warts',
 'Pink Eye',
 'COVID-19 (Coronavirus Disease 2019)',
 'COVID-19 Vaccines',
 'Fungal Infections',
 'Diphtheria',
 'E. Coli Infections',
 'Fever',
 'Genital Herpes',
 'Rubella',
 'Giardia Infections',
 'Helicobacter Pylori Infections',
 'H1N1 Flu (Swine Flu)',
 'Haemophilus Infections',
 'Mycob